# Imports

In [ ]:
#installing necessary packages
!pip install keras-tuner --upgrade
!pip install np_utils

In [ ]:
# Making Import modules necessary for hyperparameter tuning and creating CNN model
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Flatten
from keras.layers import Dropout
from keras.layers.convolutional import Conv1D
from keras.layers.convolutional import MaxPooling1D
from keras.utils.np_utils import to_categorical
from keras.utils import np_utils
from tensorflow import keras
from tensorflow.keras import layers
from keras_tuner.tuners import RandomSearch
from keras_tuner.engine.hypermodel import HyperModel
from keras_tuner.engine.hyperparameters import HyperParameters
import tensorflow as tf
import keras_tuner

In [ ]:
#Making additional imports
import pandas as pd
from numpy import mean
from numpy import std
from numpy import dstack
from pandas import read_csv
from matplotlib import pyplot
import matplotlib.pyplot as plt
import sklearn.metrics as metrics
import numpy as np
import warnings
warnings.filterwarnings('ignore')
import numpy as np
import os, sys
import re
import seaborn as sns
import sklearn.model_selection
from sklearn.model_selection import train_test_split
from sklearn import preprocessing  
from sklearn.random_projection import SparseRandomProjection
from sklearn.random_projection import GaussianRandomProjection

# Data work

In [ ]:
#load datasets
inputData = pd.read_csv("GSE59739_DataTable.txt",sep="\t") 
dataNo=1

"""
In this programm three scRNA-seq datasets are tested. The datasets should be supplied to the program one by one
Comment the tested dataset and uncomment the following datasets with their corresponding number (dataNo)
"""

#inputData = pd.read_csv("GSE103334_FPKM_CKP25_TOPHAT.txt",sep="\t")
#dataNo=2 
#inputData = pd.read_csv("GSE86469_GEO.islet.single.cell.processed.data.RSEM.raw.expected.counts.csv") 
#dataNo=3 

inputData.head(200)

In [ ]:
Data=inputData.T
new_header1 = Data.iloc[0] 
Data = Data[1:]
Data.columns = new_header1
print(Data.shape)
Data.head(500)

In [ ]:
#Split the data into train and test data -> train:test =80:20
train, test= sklearn.model_selection.train_test_split(Data, test_size=0.2, train_size=0.8, random_state=42, shuffle=True, stratify=None)

# Labeling the data

In [ ]:
def yData(data, dataNo):
    
    """
    Adds labels to the input data based on the information given in the GEO database
    """
    if dataNo==1:
        i1=data.index.shape[0]
        Y=[]
        row=0
        while row<i1:
            if data.index.str.contains('128')[row] or data.index.str.contains('129')[row] or data.index.str.contains('130')[row]:
                Y.append(0)
            elif data.index.str.contains('141')[row] or  data.index.str.contains('142')[row]:
                Y.append(1)
            elif data.index.str.contains('226')[row] or data.index.str.contains('227')[row] or data.index.str.contains('228')[row] or data.index.str.contains('281')[row] or data.index.str.contains('282')[row]:
                Y.append(2)
            row=row+1
        y=np.array(Y)   
        y=y.reshape(i1,1)
    elif dataNo==2:
        i1=data.index.shape[0]
        Y=[]
        row=0
        while row<i1:
            if data.index.str.contains('1w')[row]:
                Y.append(1)
            elif data.index.str.contains('0w')[row]:
                Y.append(0)
            elif data.index.str.contains('2w')[row]:
                Y.append(2)
            elif data.index.str.contains('6w')[row]:
                Y.append(3)
            row=row+1
        y=np.array(Y)   
        y=y.reshape(i1,1)
    elif dataNo==3:
        i1=data.index.shape[0]
        Y=[]
        row=0
        while row<i1:
            if data.index.str.contains('10th')[row] or data.index.str.contains('11th')[row] or data.index.str.contains('12th')[row] or data.index.str.contains('13th')[row]:
                Y.append(1)
            else:
                Y.append(0)
            row=row+1
        y=np.array(Y)   
        y=y.reshape(i1,1)
    return y

In [ ]:
print(train)
trainY=yData(train, dataNo)
trainY

In [ ]:
print(test)
testY=yData(test, dataNo)
testY

In [ ]:
#Random Projection
def RandomProjection(test, train, dataNo, n_components):
    """
    Performs the dimensionality reduction using Random Projection method
    Two Random Projection techniques: Sparse RP and Gaussian Rp
    n_components = final reduced dimension
    """
    rng = np.random.RandomState(42)
    transformer = SparseRandomProjection(n_components=n_components,random_state=rng)
    #transformer = GaussianRandomProjection(n_components=n_components,random_state=rng)
    train_new = transformer.fit_transform(train)
    train_norm_RP= pd.DataFrame(train_new,index=train.index)
    test_new = transformer.transform(test)
    test_norm_RP= pd.DataFrame(test_new,index=test.index)
    testy= yData(test_norm_RP, dataNo)
    trainy=yData(train_norm_RP, dataNo)
    return np.expand_dims(np.array(train_norm_RP), axis=2), trainy, np.expand_dims(np.array(test_norm_RP), axis=2), testy

In [ ]:
def Normalization(test, train, dataNo):
    if dataNo==1:
        train_n=train.iloc[:,4:25337]
        test_n=test.iloc[:,4:25337]
    else:
        train_n=train
        test_n=test
    """
    Performs Min-Max Normalization
    fit_transform to train data
    use statistic from train data to normalize the test data
    """
    x = train_n.values
    min_max_scaler = preprocessing.MinMaxScaler()
    x_scaled = min_max_scaler.fit_transform(x)
    train_norm = pd.DataFrame(x_scaled,columns = train_n.columns, index=train_n.index)
    x_test = test_n.values
    normalized_test_X = min_max_scaler.transform(x_test)
    test_norm = pd.DataFrame(normalized_test_X,columns = test_n.columns, index=test_n.index)
    print("Normalized train data",train_norm)
    print("Normalized test data",test_norm)
    return train_norm, test_norm

In [ ]:
def hcluster(trainH, testH):
    #Hierarchical clustering with average linkage
    sys.setrecursionlimit(30000)
    g_train = sns.clustermap(trainH.T,row_cluster=True,method="average")   
    #get sorted list of genes
    reordered_labels = trainH.T.index[g_train.dendrogram_row.reordered_ind].tolist()
    #reorder the test data based on clustering result
    test_out=testH.T.reindex(reordered_labels)
    test_norm_cluster=test_out.T
    #reorder the train data based on clustering result
    train_out=trainH.T.reindex(reordered_labels)
    train_norm_cluster=train_out.T

    return train_norm_cluster,test_norm_cluster


In [ ]:
train_norm, test_norm = Normalization(test, train, dataNo)

In [ ]:
Train, Test=hcluster(train_norm, test_norm)

In [ ]:
#Divide into 5 folds
if dataNo==1:
    train=train.iloc[:,4:25337]
train_folds=train.sample(frac=1)
length1 = int(len(train_folds)/5) #length of each fold
folds1 = []
for i in range(4):
    folds1 += [train_folds[i*length1:(i+1)*length1]]
folds1 += [train_folds[4*length1:len(train_folds)]]
folds=folds1

# Single

In [ ]:
f = open("model_summary.txt", "w")
from sklearn import model_selection
from keras_tuner.engine import trial
class CVTuner(keras_tuner.engine.tuner.Tuner):
    """
    5-fold CV for hyperparameter tuning 
    """
    def run_trial(self, trial, *fit_args, **fit_kwargs):
        accuracy = []
        for i in range(5):
            if i==0:
                x_train=pd.concat([folds[0], folds[1],folds[2], folds[3]])
                x_test=folds[4]
            elif i==1: 
                x_train=pd.concat([folds[0], folds[1],folds[2], folds[4]])
                x_test=folds[3]
            elif i==2: 
                x_train=pd.concat([folds[0], folds[1],folds[3], folds[4]])
                x_test=folds[2]
            elif i==3: 
                x_train=pd.concat([folds[0], folds[2],folds[3], folds[4]])
                x_test=folds[1]
            elif i==4: 
                x_train=pd.concat([folds[1], folds[2],folds[3], folds[4]])
                x_test=folds[0]
            """
            The x_train and x_test is train and validation data for one iteration of the CV 
            Both train and validation data is normalized, clustered in each CV iteration
            """
            train_normIn, test_normIn = Normalization(x_test, x_train, dataNo)
            xTrain, xTest=hcluster(train_normIn,test_normIn)
            RPdimensions=trial.hyperparameters.Choice('RP_dimension',values=[2500, 3025, 3600,4000, 4225, 4900, 5625, 6400])
            x_train_CV, y_train_CV, x_val, y_val = RandomProjection(xTest, xTrain, dataNo, RPdimensions)
            model = self.hypermodel.build(trial.hyperparameters)
            print(str(trial.hyperparameters.values))
            hist = model.fit(x_train_CV,y_train_CV,
            validation_data=(x_val,y_val),
            epochs=trial.hyperparameters.Int('epochs', 5, 100, step=5),
            batch_size=trial.hyperparameters.Int('batch_size', 32, 128, step=32),)
            accuracy.append([hist.history[k][-1] for k in hist.history]) #Accuracy of model for each CV iteration is saved 
        val_accuracy = np.asarray(accuracy) 
        f = open("model_summary.txt", "a")
        f.write("acc: "+str(val_accuracy))
        f.write(str(np.mean(val_accuracy[:,1]))) #The accuracy of the model is estimated to be average of accuracy values in 5 iteration
        f.close()
        self.oracle.update_trial(trial.trial_id, {k:np.mean(val_accuracy[:,i]) for i,k in enumerate(hist.history.keys())})
        tf.keras.models.save_model(model,trial.trial_id)
    
    
def create_model1(hp):
    """
    It is a function to create a 1D CNN model
    """
    if hp:
        #Define hyperparameters
        dropout_rate = hp.Float('dropout_rate', min_value=0.1, max_value=0.5, step= 0.1)
        output = hp.Float('output', min_value=50, max_value=300, step= 50)
        filters=hp.Choice('filters',values=[8, 16, 32, 64, 128, 256])
        kernel_size=hp.Choice('kernel_size', values = [2,3,5,7,11])
        #learning_rate = hp.Float('learning_rate', min_value=0.00001, max_value=0.1, step=0.00001)
        learning_rate=1e-4
        num_hidden_layers = hp.Choice('num_hidden_layers', values=[1, 2, 3])
        RP_dimension=hp.Choice('RP_dimension',values=[2500, 3025, 3600,4000, 4225, 4900, 5625, 6400])
    else:
        dropout_rate = 0.1
        num_units = 8
        learning_rate = 1e-4
        num_hidden_layers = 1
        filters=64
        kernel_size=11
        RP_d=3600

    x_train_CV, y_train_CV, x_val, y_val = RandomProjection(Test, Train, dataNo, RP_dimension)
    model = tf.keras.models.Sequential()
    model.add(Conv1D(filters,kernel_size, activation='relu', input_shape=(x_train_CV.shape[1], x_train_CV.shape[2])))
    if dataNo==1:
        classNo = 3
    elif dataNo==2:
        classNo=4
    else: 
        classNo=2


  
    for _ in range(0, num_hidden_layers):
        model.add(Conv1D(filters, kernel_size, activation='relu'))
        model.add(MaxPooling1D(pool_size=2))
        model.add(Dropout(dropout_rate))
        model.add(Flatten())
        model.add(Dense(output, activation='relu'))
        model.add(tf.keras.layers.Dense(classNo, activation='softmax'))

    model.compile(
      loss='sparse_categorical_crossentropy',
      #optimizer=tf.keras.optimizers.Adam(tf.cast(learning_rate, tf.float32)),
      optimizer=tf.keras.optimizers.SGD(hp.Choice('learning_rate',values=[1e-2,0.005, 1e-3, 0.0005, 1e-4]),nesterov=True),
      metrics=['accuracy']
    )   
    return model

In [ ]:
#Select the tuner class (Bayesian, Hyperband, or Random Search), the hypermodel and objective to optimize
tuner = CVTuner(
    hypermodel=create_model1,
    oracle=keras_tuner.oracles.BayesianOptimization(
    objective=keras_tuner.Objective("val_accuracy", "max"), max_trials=50),
    directory='.',
    project_name='my_projectSingle',
    #overwrite=True,
)

In [ ]:
#Start the tuner search
stop_early = tf.keras.callbacks.EarlyStopping(monitor='val_accuracy', patience=5) 
tuner.search(Train, trainY, epochs=50, validation_split=0.2, callbacks=[stop_early])
# Get the optimal hyperparameters
best_hps=tuner.get_best_hyperparameters(num_trials=10)[0]
RP_d= best_hps.get('RP_dimension')
print(f"""
The hyperparameter search is complete. The optimal number Random Projection dimension is {best_hps.get('RP_dimension')}.
""")
tuner.results_summary()

In [ ]:
#Randomly project with an optimal RP the global Train and Test data <- already normalized and hierarchically clustered 
x_train, y_train, x_test, y_test = RandomProjection(Test, Train, dataNo, RP_d)
#Build the model with best hyperparameters
model = tuner.hypermodel.build(best_hps)
#Fit the model with the dataset
model.fit(x_train, y_train, epochs=best_hps.get('epochs'), validation_split=0.2)
#Evaluate the model
eval_result = model.evaluate(x_test, y_test)
print("[test loss, test accuracy]:", eval_result)

# Ensemble

In [ ]:
#Select the tuner class (Bayesian, Hyperband, or Random Search), the hypermodel and objective to optimize
tuner = CVTuner(
    hypermodel=create_model1,
    oracle=keras_tuner.oracles.BayesianOptimization(
    objective=keras_tuner.Objective("val_accuracy", "max"), max_trials=50),
    directory='.',
    project_name='my_projectEnsemble',
    #overwrite=True,
)

In [ ]:
def hardVoting(y_classList, y_test):
  """ sums the predictions for each class label and predict the class label with the majority votes"""
  acc=0
  print('Y_classlist:', len(y_classList))
  print('Y_test:', len(y_test))
  y_hat=[None] * (len(y_test))
  for i in range(len(y_test)):
    count0=0
    count1=0
    count2=0
    count3=0
    for ii in range(len(y_classList)):
      print(y_classList[ii][i])
      if(y_classList[ii][i]==0):
        count0=count0+1
      if(y_classList[ii][i]==1):
        count1=count1+1
      if(y_classList[ii][i]==2):
        count2=count2+1
      if(y_classList[ii][i]==3):
        count3=count3+1
    classMax=max(count0, count1, count2, count3)
    if classMax==count0:
      y_hat[i]=0
    if classMax==count1:
      y_hat[i]=1
    if classMax==count2:
      y_hat[i]=2
    if classMax==count3:
      y_hat[i]=3
    if(y_hat[i]==y_test[i]):
      acc=acc+1
    print('class:',y_hat[i])
    #if(y_classList[1][i]==testY[i]):
    #  accS=accS+1
  accOut=acc/(len(y_test))
  print('Hard Voting acuracy: ',accOut)
  return accOut

In [ ]:
def softVoting(y_predictList, y_test):
  """sums the predicted probabilities for each class label and predic the class label with the largest probability"""
  y_out=[0]*(len(y_test))
  Y_out=[0]*(len(y_test))
  acc=0
  for i in range(len(y_test)):
    for ii in range(len(y_predictList)):
      y_out[i]= y_out[i]+y_predictList[ii][i]
    max_value = max(y_out[i])
    listy=y_out[i].tolist()
    max_index = listy.index(max_value)
    Y_out[i]=max_index
    if(Y_out[i]==y_test[i]):
      acc=acc+1
  print("predicted class", Y_out)
  accOut=acc/(len(y_test))
  print('Soft Voting acuracy: ',accOut)
  return accOut

In [ ]:
f = open("model_summary.txt", "w")
L=[5,10,20,30,40,50,60,70,80,90,100] #number of base classifiers
#L=[30,35,40,45,50]
resultsMajV =[]
resultsSoft=[]
for i in L:
  l=i
  y_predictList=[]
  y_classList=[]
  for ii in range(l):
    # train l number of base classifiers
    stop_early = tf.keras.callbacks.EarlyStopping(monitor='val_accuracy', patience=5) 
    tuner.search(Train, trainY, epochs=50, validation_split=0.2, callbacks=[stop_early])
    best_hps=tuner.get_best_hyperparameters(num_trials=10)[0]
    RP_d= best_hps.get('RP_dimension')
    x_train, y_train, x_test, y_test = RandomProjection(Test, Train, dataNo, RP_d)
    model = tuner.hypermodel.build(best_hps)
    model.fit(x_train, y_train, epochs=best_hps.get('epochs'), validation_split=0.2)
    Y= model.predict(x_test)
    y_classes = Y.argmax(axis=-1)
    y_predictList.append(Y)       # Save class probability values for each base classifier
    y_classList.append(y_classes) #Save class labels for each base classifier
    if l==len(y_predictList):     #Calls softVoting and hardVoting functions   
      accMajVote= hardVoting(y_classList, y_test)
      accSoft= softVoting(y_predictList, y_test)
      resultsMajV.append(accMajVote)
      resultsSoft.append(accSoft) 
      #Saves the results for majority and soft voting in a file
      f = open("model_summary.txt", "a")
      f.write("\n For l dimensions:"+ str(l))
      f.write("\n SoftVoting acc: "+str(accSoft))
      f.write("\nHard acc: "+str(accMajVote))
      f.close()